<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-01 10:27:22
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20


-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.36 C
-------------------
Today PnL: -22.65 K (-0.17%)
Current PnL: -13.78 L (-9.6%)
CY Booked + Current PnL: -7.04 L (-4.91%)
-------------------
Total profit:  4.35 L
Total loss:  -18.13 L
-------------------
Total Booked + Current PnL: 19.94 L (17.11%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.95%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 72.21 L (53.05%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.59%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,78.12,73675.0,39226.0,722.0,-0.97,113.87,0.98,115.97,NaN,54.33,0.54,12.66,AR,ATH,ELECTRICAL
16,BLUSPRING,226.45,84.55,NaN,NaN,4.17,16348.0,-33245.0,2168.0,-1.70,-67.04,13.26,-62.66,NaN,-15.33,0.12,9.63,XY24,ATH,MISC
28,DIGITIDE,188.38,237.72,NaN,NaN,1.04,42968.0,1713.0,9092.0,0.04,4.15,21.16,26.19,1.0,0.19,0.32,0.04,XY24,ATH,IT
3,ADANIPORTS,1103.69,1583.00,2.03,M-LC,27.08,102517.0,24155.0,9872.0,-0.43,30.82,9.63,43.43,77.0,2.45,0.75,37.78,XY24,NTT,MISC
87,UJJIVANSFB,52.77,53.00,51.69,M-SC,88.54,131193.0,-11286.0,11912.0,-0.90,-7.92,9.08,0.44,243.0,-0.95,0.96,54.84,OX40N,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,87.81,M-MC,35.42,225014.0,52.0,167613.0,-1.00,0.02,74.49,74.53,185.0,0.00,1.65,30.64,XY24,BTT,STEEL
12,BANKINDIA,116.91,190.00,-31.29,M-MC,18.75,181130.0,1322.0,111087.0,-0.67,0.74,61.33,62.52,171.0,0.01,1.33,35.07,XR,NTT,BANKS
55,LALPATHLAB,2804.19,3545.00,-33.29,H-SC,44.79,153439.0,-791.0,41536.0,-0.23,-0.51,27.07,26.42,125.0,-0.02,1.13,21.00,X40N,NTT,HEALTHCARE
15,BLUESTARCO,1646.70,2337.55,-11.71,H-SC,76.04,165700.0,1030.0,68053.0,1.31,0.63,41.07,41.95,121.0,0.02,1.22,8.14,X40N,ATH,AC
60,NATIONALUM,189.63,247.44,-50.15,H-MC,63.54,100064.0,-440.0,31080.0,-2.02,-0.44,31.06,30.49,97.0,-0.01,0.74,31.83,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,6904.43,8285.00,-17.74,H-LC,79.17,112132.0,8566.0,12144.0,3.22,8.27,10.83,20.00,30.0,0.71,0.82,23.51,X40N,BTT,HEALTHCARE
74,SIEMENS,4406.90,7969.85,-1.34,X-LC,46.88,83340.0,-26832.0,115909.0,2.52,-24.35,139.08,80.85,14.0,-0.23,0.61,22.75,AR,ATH,ELECTRICAL
54,KPIGREEN,497.21,732.16,29.10,H-SC,71.88,131998.0,6701.0,52509.0,1.78,5.35,39.78,47.25,139.0,0.13,0.97,65.14,MH,ATH,POWER
30,DMART,3764.92,5201.00,-13.22,X-LC,90.62,217869.0,33388.0,36972.0,1.69,18.10,16.97,38.14,15.0,0.90,1.60,30.62,X40N,NTT,FMCG
84,TITAGARH,1097.23,1548.00,2.28,H-SC,67.71,167624.0,-24391.0,103273.0,1.59,-12.70,61.61,41.08,152.0,-0.24,1.23,41.87,XY24,NTT,ENGINEERING


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,DIXON,14367.00,18940.15,-36.69,H-MC,32.29,116512.0,1576.0,35012.0,-2.80,1.37,30.05,31.83,99.0,0.05,0.86,33.54,X40N,ATH,IT
60,NATIONALUM,189.63,247.44,-50.15,H-MC,63.54,100064.0,-440.0,31080.0,-2.02,-0.44,31.06,30.49,97.0,-0.01,0.74,31.83,MH,ATH,METALS
4,ALKYLAMINE,2347.98,4546.37,7.43,X-SC,98.96,99975.0,-988.0,95516.0,-1.82,-0.98,95.54,93.63,64.0,-0.01,0.73,48.91,SR,ATH,CHEMICALS
16,BLUSPRING,226.45,84.55,NaN,NaN,4.17,16348.0,-33245.0,2168.0,-1.70,-67.04,13.26,-62.66,NaN,-15.33,0.12,9.63,XY24,ATH,MISC
70,SATIN,199.77,274.00,-17.21,H-SC,83.33,163749.0,-32225.0,105045.0,-1.62,-16.44,64.15,37.16,130.0,-0.31,1.20,22.29,XY24,NTT,FINANCE


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,51.69,M-SC,88.54,131193.0,-11286.0,11912.0,-0.90,-7.92,9.08,0.44,243.0,-0.95,0.96,54.84,OX40N,NTT,BANKS
92,VIPIND,488.80,489.0,-214.88,H-SC,97.92,82885.0,-12431.0,12474.0,0.93,-13.04,15.05,0.04,154.0,-1.00,0.61,63.48,OX40N,NTT,MISC
81,TATAELXSI,7332.28,7332.0,-16.30,X-MC,29.17,87906.0,-14746.0,14742.0,-0.49,-14.36,16.77,-0.00,58.0,-1.00,0.65,33.63,OX40N,NTT,IT
53,KANSAINER,299.63,340.0,-67.65,H-SC,31.25,224865.0,-44802.0,81131.0,-0.91,-16.61,36.08,13.47,142.0,-0.55,1.65,13.14,XY24,NTT,PAINTS
46,INDIGOPNTS,1407.73,1408.0,120.98,H-SC,87.50,144832.0,-29727.0,29763.0,0.50,-17.03,20.55,0.02,127.0,-1.00,1.06,25.59,OX40N,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,1.04,42968.0,1713.0,9092.0,0.04,4.15,21.16,26.19,1.0,0.19,0.32,0.04,XY24,ATH,IT
83,TCS,3794.03,4998.00,-17.10,X-LC,41.67,270036.0,-25898.0,119815.0,0.61,-8.75,44.37,31.73,1.0,-0.22,1.98,8.09,X40,BTT,IT
48,INFY,1461.46,2275.00,-10.27,X-LC,58.33,288029.0,26428.0,119186.0,0.46,10.10,41.38,55.67,2.0,0.22,2.12,16.82,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-17.84,X-LC,21.88,257589.0,-12758.0,47396.0,0.23,-4.72,18.40,12.81,6.0,-0.27,1.89,7.73,XY25,NTT,FMCG
61,NESTLEIND,2268.60,2755.00,-9.17,X-LC,51.04,288066.0,22640.0,34280.0,-0.13,8.53,11.90,21.44,9.0,0.66,2.12,15.25,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.0,-1.39,M-LC,25.52,172051.0,31211.0,13282.0,-1.48,22.16,7.72,31.59,89.0,2.35,1.26,49.45,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,2.03,M-LC,27.08,102517.0,24155.0,9872.0,-0.43,30.82,9.63,43.43,77.0,2.45,0.75,37.78,XY24,NTT,MISC
32,ENRIN,1377.95,2975.9,NaN,NaN,78.12,73675.0,39226.0,722.0,-0.97,113.87,0.98,115.97,NaN,54.33,0.54,12.66,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,87.81,M-MC,35.42,225014.0,52.0,167613.0,-1.00,0.02,74.49,74.53,185.0,0.00,1.65,30.64,XY24,BTT,STEEL
54,KPIGREEN,497.21,732.16,29.10,H-SC,71.88,131998.0,6701.0,52509.0,1.78,5.35,39.78,47.25,139.0,0.13,0.97,65.14,MH,ATH,POWER
20,CAMS,3643.00,5250.99,6.54,H-SC,57.29,119616.0,17612.0,27416.0,-0.42,17.27,22.92,44.14,124.0,0.64,0.88,36.31,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,2.03,M-LC,27.08,102517.0,24155.0,9872.0,-0.43,30.82,9.63,43.43,77.0,2.45,0.75,37.78,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,-1.39,M-LC,25.52,172051.0,31211.0,13282.0,-1.48,22.16,7.72,31.59,89.0,2.35,1.26,49.45,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,78.12,73675.0,39226.0,722.0,-0.97,113.87,0.98,115.97,NaN,54.33,0.54,12.66,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-38.30,M-SC,91.67,203477.0,31856.0,130205.0,-1.23,18.56,63.99,94.43,198.0,0.24,1.50,50.18,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-3.77,M-LC,77.08,164672.0,13727.0,95724.0,-0.14,9.09,58.13,72.51,69.0,0.14,1.21,13.43,XR,NTT,IT
54,KPIGREEN,497.21,732.16,29.10,H-SC,71.88,131998.0,6701.0,52509.0,1.78,5.35,39.78,47.25,139.0,0.13,0.97,65.14,MH,ATH,POWER
45,INDIAMART,2327.09,4871.06,-47.14,H-SC,92.71,137503.0,14167.0,120659.0,-0.14,11.49,87.75,109.32,122.0,0.12,1.01,35.84,AR,ATH,MISC


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-51.71,L-SC,100.00,72576.0,-40973.0,81024.0,0.93,-36.08,111.64,35.27,261.0,-0.51,0.53,77.63,XR,NTT,HOTELS
32,ENRIN,1377.95,2975.90,NaN,NaN,78.12,73675.0,39226.0,722.0,-0.97,113.87,0.98,115.97,NaN,54.33,0.54,12.66,AR,ATH,ELECTRICAL
58,MASFIN,326.60,402.39,-19.15,H-SC,60.42,91740.0,-6240.0,28981.0,-0.73,-6.37,31.59,23.21,137.0,-0.22,0.67,33.05,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,7.43,X-SC,98.96,99975.0,-988.0,95516.0,-1.82,-0.98,95.54,93.63,64.0,-0.01,0.73,48.91,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,2.03,M-LC,27.08,102517.0,24155.0,9872.0,-0.43,30.82,9.63,43.43,77.0,2.45,0.75,37.78,XY24,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-51.71,L-SC,100.00,72576.0,-40973.0,81024.0,0.93,-36.08,111.64,35.27,261.0,-0.51,0.53,77.63,XR,NTT,HOTELS
4,ALKYLAMINE,2347.98,4546.37,7.43,X-SC,98.96,99975.0,-988.0,95516.0,-1.82,-0.98,95.54,93.63,64.0,-0.01,0.73,48.91,SR,ATH,CHEMICALS
92,VIPIND,488.80,489.00,-214.88,H-SC,97.92,82885.0,-12431.0,12474.0,0.93,-13.04,15.05,0.04,154.0,-1.00,0.61,63.48,OX40N,NTT,MISC
52,JPPOWER,18.73,26.20,-31.42,L-SC,96.88,137753.0,-4089.0,60666.0,-0.60,-2.88,44.04,39.88,259.0,-0.07,1.01,39.60,XY24,NTT,POWER
72,SFL,1016.35,1287.00,41.13,H-SC,95.83,186250.0,-75968.0,145796.0,-0.65,-28.97,78.28,26.63,144.0,-0.52,1.37,15.80,XY24,NTT,MISC


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.00
1,25,42.05
2,50,70.44


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.45
LC    34.22
MC    21.37
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.44
X40      12.27
XY25     12.11
X40N     11.79
XR       10.14
AR        8.30
OX40N     8.16
MH        1.71
X5K       1.47
X200      1.41
SR        1.22
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.65
M-SC    15.98
H-LC    14.22
X-LC    12.29
H-MC     9.46
X-MC     7.54
M-LC     6.64
X-SC     4.37
M-MC     4.01
L-SC     2.45
L-LC     1.07
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.21,-1.17,35.00
IT,12.12,-10.39,64.59
FINANCE,7.93,-11.72,52.16
BANKS,7.90,-5.89,49.10
MISC,6.66,-14.79,68.05
PAINTS,5.65,-13.40,35.75
ELECTRICAL,5.53,2.10,43.63
HEALTHCARE,4.52,-4.86,35.50
AUTO,4.33,-14.65,65.07


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2723753.0,27
XR,1079176.0,13
AR,1014970.0,10
X40,650080.0,9
X40N,560681.0,11
XY25,472851.0,8
OX40N,321156.0,11
SR,177043.0,2
MH,83589.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1975286.0,20
M-SC,1770161.0,21
H-LC,607088.0,12
H-MC,548700.0,8
X-LC,502678.0,7
X-MC,426888.0,6
M-MC,423610.0,3
X-SC,324688.0,5
M-LC,268944.0,5


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      858720.0      6
M-SC       XY24      815956.0      7
H-SC       AR        520861.0      3
M-SC       XR        334532.0      4
M-MC       XY24      312523.0      2
H-MC       XY24      300629.0      3
X-LC       X40       239001.0      2
H-SC       X40N      232134.0      4
           XR        218677.0      3
H-LC       X40N      198126.0      4
M-SC       AR        181479.0      2
           OX40N     173597.0      5
M-LC       XY24      160234.0      3
H-LC       X40       157171.0      3
X-MC       X40       151831.0      2
L-SC       XR        142783.0      2
X-SC       XR        123391.0      1
X-LC       AR        115909.0      1
M-MC       XR        111087.0      1
X-LC       XY25      110796.0      3
M-SC       XY25      110576.0      1
X-MC       XY25      104998.0      1
           XY24       96880.0      1
M-LC       XR         95724.0      1
X-SC       SR         95516.0      1
H-SC       OX40N      92385.0      3
H-MC       XY25       89119.0      1
M-SC       SR         81527.0      1
H-LC       AR         72770.0      1
M-SC       X40        72494.0      1
H-LC       X200       69098.0      1
           X5K        68387.0      2
X-SC       XY24       65349.0      1
H-MC       AR         63277.0      1
L-SC       XY24       60666.0      1
           AR         59952.0      1
X-MC       X40N       58437.0      1
L-MC       XR         52982.0      1
H-SC       MH         52509.0      1
L-LC       XY25       44376.0      1
H-LC       XY24       41536.0      1
X-SC       OX40N      40432.0      2
X-LC       X40N       36972.0      1
H-MC       X40N       35012.0      1
           MH         31080.0      1
           X40        29583.0      1
X-MC       OX40N      14742.0      1
M-LC       XY25       12986.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 53.0 seconds
